In [21]:
import pandas as pd
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
cider_scorer = Cider()
bleu_scorer = Bleu(4)
meteor_scorer = Meteor()
rouge_scorer = Rouge()

In [22]:
!ls ../../model_cocopretrain_iotfinetune/*.csv

../../model_cocopretrain_iotfinetune/model_resnet152_1.csv
../../model_cocopretrain_iotfinetune/model_resnet152_5.csv


In [23]:
import pandas as pd

# Define file paths and their corresponding column names in a dictionary (flipped)
file_paths = {
    'vgg19': 'model_vgg19_10_prediction_IMERIT_2024_03_25_TEST_v5.csv',
    'rn152': 'model_resnet152_10_prediction_IMERIT_2024_03_25_TEST_v5.csv',
    'target': 'metadata_IMERIT_2024_03_25_TEST_v5.csv',
    'rn152_ft_highlr_1': '../../model_cocopretrain_iotfinetune/model_resnet152_1.csv',
    'rn152_ft_highlr_5': '../../model_cocopretrain_iotfinetune/model_resnet152_5.csv',
    'rn152_ft_1': '../../model_cocopretrain_iotfinetune_2/model_resnet152_1.csv',
    'rn152_ft_5': '../../model_cocopretrain_iotfinetune_2/model_resnet152_5.csv',
    'rn152_ft_10': '../../model_cocopretrain_iotfinetune_2/model_resnet152_10.csv',
}

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through the file paths and load the CSV files
for column_name, path in file_paths.items():
    df = pd.read_csv(path).rename(columns={'text': column_name})
    dataframes.append(df)

# Start merging with the first dataframe
merged_df = dataframes[0]

# Loop through the rest of the dataframes and merge them on 'file_name'
for df in dataframes[1:]:
    merged_df = pd.merge(merged_df, df, on='file_name')

# Now `merged_df` contains the merged data from all the CSV files


In [24]:
merged_df 
merged_df.to_csv("caption_comparison.csv")

In [25]:
for model_name in file_paths.keys():
    print('-'*20, model_name.upper(), '-'*20)
    refer_captions = {}
    predc_captions = {}
    for i in range(len(merged_df)):
        refer_captions[merged_df.iloc[i]['file_name']] = [str(merged_df.iloc[i][model_name])]
        predc_captions[merged_df.iloc[i]['file_name']] = [str(merged_df.iloc[i]['target'])]
    c, _ = cider_scorer.compute_score(refer_captions, predc_captions); print("CIDEr", c)
    b, _ = bleu_scorer.compute_score(refer_captions, predc_captions); print('BLEU', b)
    r, _ = rouge_scorer.compute_score(refer_captions, predc_captions); print('ROUGE', r, '\n')

-------------------- VGG19 --------------------
CIDEr 0.058380726233160894
{'testlen': 440, 'reflen': 737, 'guess': [440, 380, 320, 260], 'correct': [56, 3, 0, 0]}
ratio: 0.5970149253723243
BLEU [0.06480172625884807, 0.01613941277325973, 7.455765409991548e-08, 1.6878686441854953e-10]
ROUGE 0.09194621172081302 

-------------------- RN152 --------------------
CIDEr 0.031495960447011424
{'testlen': 440, 'reflen': 722, 'guess': [440, 380, 320, 260], 'correct': [55, 3, 0, 0]}
ratio: 0.6094182825476324
BLEU [0.06585166063936791, 0.016549335255775628, 7.668126745747093e-08, 1.7385524791002788e-10]
ROUGE 0.09237822120610485 

-------------------- TARGET --------------------
CIDEr 10.0
{'testlen': 440, 'reflen': 440, 'guess': [440, 380, 320, 260], 'correct': [440, 380, 320, 260]}
ratio: 0.9999999999977273
BLEU [0.9999999999954544, 0.999999999995275, 0.9999999999950507, 0.9999999999947583]
ROUGE 1.0 

-------------------- RN152_FT_HIGHLR_1 --------------------
CIDEr 0.045983458500352646
{'testl